# Hotel


In [ ]:
# !pip install opendatasets
# !pip install pandas
# !pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import opendatasets as od
# od.download(
#     "https://www.kaggle.com/competitions/hotel-id-to-combat-human-trafficking-2022-fgvc9/data")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 
Your Kaggle username: 
Your Kaggle username: 

Abort: ignored

In [ ]:
from google.colab import files,drive

drive.mount("/content/gdrive/")  

Mounted at /content/gdrive/


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import optimizers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from random import sample
from keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.models import load_model

In [ ]:
IMG_SIZE = 256

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')


In [ ]:
base_dir = 'D:/CNN Hotel Classification'
train_dir = os.path.join(base_dir,'train_images')
save_directory = os.path.join(base_dir,'processed_images10')

In [ ]:
directory_list = os.listdir(train_dir)
classes = np.empty((len(directory_list),1))

for i, dir in enumerate(directory_list):
  temp_dir = os.path.join(train_dir, dir)
  classes[i] = len(os.listdir(temp_dir))

plt.boxplot(classes)

204361 11
Max 15
Total 40361
Media 14
Min 11


In [ ]:
directory_list = os.listdir(train_dir)

min_img = 10
target_number = 50
max_target = 600

if not os.path.exists(save_directory):
  os.mkdir(save_directory)

for dir in directory_list:
  temp_dir = os.path.join(train_dir, dir)
  if len(os.listdir(temp_dir)) < min_img:
    continue
  
  temp_save_dir = os.path.join(save_directory, dir)
  if not os.path.exists(temp_save_dir):
    os.mkdir(temp_save_dir)

  img_names = [os.path.join(temp_dir, name) for name in os.listdir(temp_dir)]
  # Oversampling
  if len(img_names) < target_number:
    remainder = sample(img_names, k= target_number % len(img_names)) 
    target_per_img = target_number // len(img_names) - 1
    i, j = 0, 0

    for img_path in img_names:
      name = "0" * (1-i//10) + str(i)
      name = os.path.join(temp_save_dir, name + '.jpg')
      img = tf.keras.preprocessing.image.load_img(img_path).resize((IMG_SIZE, IMG_SIZE))
      img.save(name)
      i += 1
      if target_per_img > 0 or img_path in remainder:
        img = tf.keras.preprocessing.image.load_img(img_path).resize((IMG_SIZE, IMG_SIZE))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)

        for batch in train_datagen.flow(img_array, batch_size=1):
          temp = tf.keras.preprocessing.image.array_to_img(batch[0])
          name = "0" * (1-i//10) + str(i)
          name = os.path.join(temp_save_dir, name + '.jpg')
          temp.save(name)
          i += 1
          j += 1

          if (img_path in remainder and j == (target_per_img + 1)) or \
            (img_path not in remainder and j == target_per_img):
              j = 0
              break

  # Undersampling
  elif len(img_names) >= target_number:
    target = 0
    if max_target < len(img_names):
      target = max_target
    else:
      target = len(img_names)
    random_images = sample(img_names, k= target)
    for i, img_path in enumerate(random_images):
      name = "0" * (1-i//10) + str(i)
      name = os.path.join(temp_save_dir, name + '.jpg')
      img = tf.keras.preprocessing.image.load_img(img_path).resize((IMG_SIZE, IMG_SIZE))
      img.save(name)
    


In [ ]:
train_data = ImageDataGenerator(rescale=1/255,
                                samplewise_center=True,
                                validation_split=0.1)

train_generator = train_data.flow_from_directory(train_dir, 
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size=128,
                                                 class_mode="categorical",
                                                 subset='training')

valid_generator = train_data.flow_from_directory(train_dir, 
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size=128,
                                                 class_mode="categorical",
                                                 subset='validation')


Found 49346 images belonging to 2872 classes.
Found 11977 images belonging to 2872 classes.


In [ ]:
conv_base = ResNet50V2(weights='imagenet',
                       include_top=False,
                       input_shape=(IMG_SIZE, IMG_SIZE, 3))
conv_base.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

In [ ]:
conv_base.trainable = False
model = Sequential([
    conv_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(3116, activation='softmax')                    
])
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['acc', tf.keras.metrics.TopKCategoricalAccuracy(k=5)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dense_1 (Dense)             (None, 2872)              738104    
                                                                 
Total params: 23,841,656
Trainable params: 23,841,656
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=11, validation_data=valid_generator, validation_steps=len(valid_generator))

Epoch 1/30
100/100 [==============================] - 826s 8s/step - loss: 7.7910 - acc: 0.0245 - val_loss: 7.7583 - val_acc: 0.0250
Epoch 2/30
100/100 [==============================] - 790s 8s/step - loss: 7.7101 - acc: 0.0230 - val_loss: 7.7610 - val_acc: 0.0190
Epoch 3/30
100/100 [==============================] - 731s 7s/step - loss: 7.6814 - acc: 0.0215 - val_loss: 7.7038 - val_acc: 0.0290
Epoch 4/30
100/100 [==============================] - 761s 8s/step - loss: 7.6834 - acc: 0.0220 - val_loss: 7.6962 - val_acc: 0.0230
Epoch 5/30
100/100 [==============================] - 732s 7s/step - loss: 7.6762 - acc: 0.0245 - val_loss: 7.6490 - val_acc: 0.0230
Epoch 6/30
100/100 [==============================] - 682s 7s/step - loss: 7.6233 - acc: 0.0230 - val_loss: 7.6025 - val_acc: 0.0270
Epoch 7/30
100/100 [==============================] - 667s 7s/step - loss: 7.6151 - acc: 0.0225 - val_loss: 7.5519 - val_acc: 0.0330
Epoch 8/30
100/100 [==============================] - 636s 6s/step - 

In [ ]:
from tensorflow.keras.models import load_model

model.save('model11epch.h5')

In [ ]:
plt.subplot(1, 3, 1)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Validation")
plt.legend()

plt.subplot(1, 3, 2)
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Categorical Accuracy")
plt.plot(history.history["acc"], label="Train")
plt.plot(history.history["val_acc"], label="Validation")
plt.legend()

plt.subplot(1, 3, 3)
plt.title("Accuracy Top 5")
plt.xlabel("Epochs")
plt.ylabel("Categorical Accuracy")
plt.plot(history.history["top_k_categorical_accuracy"], label="Train")
plt.plot(history.history["val_top_k_categorical_accuracy"], label="Validation")
plt.legend()

In [ ]:
trained_model = load_model('model11epch.h5')

# Fine Tuning

In [ ]:
base_ResNet50V2 = trained_model.layers[0]

for layer in base_ResNet50V2.layers[152:]:
    layer.trainable = True

# Make sure you have unfrozen the correct layers
for i, layer in enumerate(base_ResNet50V2.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
trained_model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.00001), metrics=['acc', tf.keras.metrics.TopKCategoricalAccuracy(k=5)])

In [ ]:
history2 = trained_model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=valid_generator, validation_steps=len(valid_generator))

In [ ]:
plt.subplot(1, 3, 1)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history["loss"], label="Train")
plt.plot(history.history["val_loss"], label="Validation")
plt.legend()

plt.subplot(1, 3, 2)
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Categorical Accuracy")
plt.plot(history.history["acc"], label="Train")
plt.plot(history.history["val_acc"], label="Validation")
plt.legend()

plt.subplot(1, 3, 3)
plt.title("Accuracy Top 5")
plt.xlabel("Epochs")
plt.ylabel("Categorical Accuracy")
plt.plot(history.history["top_k_categorical_accuracy"], label="Train")
plt.plot(history.history["val_top_k_categorical_accuracy"], label="Validation")
plt.legend()

In [ ]:
model.save('modelFT10.h5')